<div align="center"> 
    <hr>
  <font size="6"><b>Dev | Clean-Clean Entity Resolution Notebook <br>with pre-trained embeddings and FAISS</b> </font>
   <hr>
</div>

In [1]:
!python --version

Python 3.9.1


In [1]:
import os
import sys
import pandas as pd
import networkx
from networkx import draw, Graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
from pyjedai.utils import (
    text_cleaning_method,
    print_clusters,
    print_blocks,
    print_candidate_pairs
)
from pyjedai.evaluation import Evaluation, write
from pyjedai.datamodel import Data
d1 = pd.read_csv("./data/ccer/D2/abt.csv", sep='|', engine='python', na_filter=False).astype(str)
d2 = pd.read_csv("./data/ccer/D2/buy.csv", sep='|', engine='python', na_filter=False).astype(str)
gt = pd.read_csv("./data/ccer/D2/gt.csv", sep='|', engine='python')

data = Data(dataset_1=d1,
            attributes_1=['id','name','description'],
            id_column_name_1='id',
            dataset_2=d2,
            attributes_2=['id','name','description'],
            id_column_name_2='id',
            ground_truth=gt)

# Block Building

In [2]:
from pyjedai.vector_based_blocking import EmbeddingsNNBlockBuilding

In [3]:
emb = EmbeddingsNNBlockBuilding(vectorizer='roberta',
                                similarity_search='faiss')
blocks = emb.build_blocks(data)

Embeddings-NN Block Building:   0%|          | 0/2152 [00:00<?, ?it/s]

Device selected:  cpu


In [ ]:
emb.evaluate(blocks, with_classification_report=True)

# Entity Matching

In [ ]:
from pyjedai.matching import EntityMatching

In [ ]:
EM = EntityMatching(metric='cosine_vector_similarity')

pairs_graph = EM.predict(blocks,
                         data,
                         tqdm_disable=True,
                         vectors_d1=emb.vectors_1,
                         vectors_d2=emb.vectors_2)

In [ ]:
draw(pairs_graph)

In [ ]:
EM.evaluate(pairs_graph)

In [ ]:
EM.plot_distribution_of_scores()

# Entity Clustering

In [ ]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering

In [ ]:
ccc = ConnectedComponentsClustering()
clusters = ccc.process(pairs_graph, data)

In [ ]:
ccc.report()

In [ ]:
ccc.evaluate(clusters, with_classification_report=True)

In [ ]:
from pyjedai.clustering import ConnectedComponentsClustering, UniqueMappingClustering
umc = UniqueMappingClustering()
clusters = umc.process(pairs_graph, data)

In [ ]:
umc.evaluate(clusters, with_classification_report=True)